In [32]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [33]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

In [34]:
text = np.genfromtxt("imat2009_train_new.txt",delimiter='\t',dtype=None, encoding=None)
arr = np.arange(0, 247)
dataFrames = []
for line in text:
    info = line.split(" ")
    relevance = float(info[0])
    queryId = int(info[-1])
    info.pop(0)
    info.pop()
    info.pop()
    finalArr = [queryId, relevance]
    featureArr = np.zeros(245)
    for feature in info:
        featureInfo = feature.split(":")
        featureInfo[0] = int(featureInfo[0])
        featureInfo[1] = float(featureInfo[1])
        featureArr[featureInfo[0]-1] = featureInfo[1]
    # print(featureArr)
    featureArr = np.concatenate((finalArr, featureArr), axis = 0)
    numpArr = np.array(featureArr)
    otherArr = np.expand_dims(numpArr, axis=0)
    # print(otherArr.shape)
    df = pd.DataFrame(otherArr, columns = arr)
    dataFrames.append(df)
df = pd.concat(dataFrames)

In [35]:
df = df.astype({0:'int'})
df = df.reset_index()
df = df.drop(columns = [ 'index'])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
0,3382,1.0,0.000023,0.0,0.000000,0.000000,0.0,0.0,0.704953,0.550315,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.0,0.000023,1.000000,0.000023
1,3382,1.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.273423,0.000000,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.862745,0.000000
2,3382,1.0,0.000000,0.0,0.006800,0.051546,0.0,0.0,0.671346,0.000000,...,0.032749,0.0,0.0,0.0,0.154346,0.0,0.0,0.000000,0.811765,0.000000
3,3382,1.0,0.000000,0.0,0.000862,0.030928,0.0,0.0,0.573946,0.000000,...,0.032749,0.0,0.0,0.0,0.039509,0.0,0.0,0.000000,1.000000,0.000000
4,3382,1.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.261436,0.000000,...,0.032749,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.882353,0.000000


In [36]:
def returnPool(dataFrame):
  query = np.array(dataFrame[0])
  relevance = np.array(dataFrame[1])
  dataFrame = dataFrame.drop(columns = [0,1])
  features = np.array(dataFrame)
  obj = Pool(
    data=features,
    label=relevance,
    group_id=query
  )
  return obj

In [37]:
def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)
    
    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]
    
    return weights

def returnPoolWithWeights(dataFrame):
  query = np.array(dataFrame[0])
  relevance = np.array(dataFrame[1])
  dataFrame = dataFrame.drop(columns = [0,1])
  features = np.array(dataFrame)
  obj = Pool(
    data=features,
    label=relevance,
    group_weight=create_weights(query),
    group_id=query
  )
  return obj

In [38]:
dfs = np.split(df, [55000], axis=0)
train = returnPool(dfs[0])
test = returnPool(dfs[1])

In [39]:
default_parameters = {
    'iterations': 1000,
    'custom_metric': ['NDCG'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [40]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [45]:
model = fit_model('RMSE', {'custom_metric': ['NDCG:top=1', 'PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})

/usr/local/lib/python3.8/dist-packages/catboost/core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
train_with_weights = returnPoolWithWeights(dfs[0])
test_with_weights = returnPoolWithWeights(dfs[1])

fit_model(
    'RMSE', 
    additional_params={'train_dir': 'RMSE_weigths'}, 
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

In [43]:
PairLogit = fit_model('PairLogit')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
PairLogit.best_score_

In [44]:
YetiRank = fit_model('YetiRank')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [ ]:
YetiRank.best_score_